In [17]:
import cv2
import numpy as np

In [18]:
dim=(768,768)
left=cv2.imread('img/restaurant1.jpg',cv2.IMREAD_COLOR)
left=cv2.resize(left,dim,interpolation = cv2.INTER_AREA)  
right=cv2.imread('img/restaurant2.jpg',cv2.IMREAD_COLOR)
right=cv2.resize(right,dim,interpolation = cv2.INTER_AREA) 

images=[]
images.append(left)
images.append(right)

In [19]:
stitcher = cv2.createStitcher()

ret,pano = stitcher.stitch(images)

if ret==cv2.STITCHER_OK:
    cv2.imshow('Panorama',pano)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

img1 = cv2.imread('img/restaurant1.jpg')
img2 = cv2.imread('img/restaurant2.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# ORB, BF-Hamming 로 knnMatch  ---①
detector = cv2.ORB_create()
kp1, desc1 = detector.detectAndCompute(gray1, None)
kp2, desc2 = detector.detectAndCompute(gray2, None)

matcher = cv2.BFMatcher(cv2.NORM_HAMMING2)
matches = matcher.knnMatch(desc1, desc2, 2)


In [2]:
# 이웃 거리의 75%로 좋은 매칭점 추출---②
ratio = 0.75
good_matches = [first for first,second in matches \
                    if first.distance < second.distance * ratio]
print('good matches:%d/%d' %(len(good_matches),len(matches)))

good matches:68/500


In [3]:

# 좋은 매칭점의 queryIdx로 원본 영상의 좌표 구하기 ---③
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ])
dst_pts = np.float32([ kp2[m.queryIdx].pt for m in good_matches ])


In [4]:

#위의 코드가 이해 안되면 출력해보기
#print(kp1[m.queryIdx]) #객체
#print(kp1[m.queryIdx].pt) #좌표
    
# 원근 변환 행렬 구하기 ---⑤
mtrx, mask = cv2.findHomography( dst_pts,src_pts)

# 원본 영상 크기로 변환 영역 좌표 생성 ---⑥
hl, wl = img1.shape[:2]
hr, wr = img2.shape[:2]
w = wl + wr 
h = hl + hr

In [5]:
w

1280

In [6]:

# h,w, = img1.shape[:2]
pts = np.float32([ [[0,0]],[[0,h-1]],[[wl-1,hl-1]],[[wl-1,0]] ])

# 원본 영상 좌표를 원근 변환  ---⑦
dst = cv2.perspectiveTransform(pts,mtrx)
# # 변환 좌표 영역을 대상 영상에 그리기 ---⑧
#img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)


#원근 변환 행렬로 오른쪽 사진을 원근 변환, 결과 이미지 크기는 사진 2장 크기
dst = cv2.warpPerspective(img2, mtrx, (wr, hr))

# 왼쪽 사진을 원근 변환한 왼쪽 영역에 합성
dst[0:hl, 0:wl] = img1

# # 좋은 매칭 그려서 출력 ---⑨
res = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, \
                     flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)



cv2.imshow("Panorama", res)
cv2.waitKey(0)                        
cv2.destroyAllWindows()

In [7]:
cv2.imshow('a',dst)

In [ ]:
import cv2, numpy as np
import matplotlib.pyplot as plt
# 1. 이미지 입력 및 그레이스케일 변환
img1 = cv2.imread('img/restaurant1.jpg')                    # img1 입력
img2 = cv2.imread('img/restaurant2.jpg')                    # img2 입력
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)              # img1 gray 변환
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)              # img2 gray 변환

# 2. ORB, BF-Hamming 로 knnMatch
detector = cv2.ORB_create()                                 # ORB 검출기 생성
kp1, desc1 = detector.detectAndCompute(gray1, None)         # gray1의 키포인트와 디스크립터 검출
kp2, desc2 = detector.detectAndCompute(gray2, None)         # gray2의 키포인트와 디스크립터 검출
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2)                  # BF-Hamming 매칭 생성
matches = matcher.knnMatch(desc1, desc2, 2)                 # knnMatch, k=2(2번째 이웃 거리)

# 3. 좋은 매칭점 추출
ratio = 0.65                                                #이웃 거리의 75%로(65~75%)
good_matches = [first for first,second in matches \
                    if first.distance < second.distance * ratio]
print('good matches:%d/%d' %(len(good_matches),len(matches)))

# 4. 좋은 매칭점의 queryIdx로 원본 영상의 좌표 구하기
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ])
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches ])

mtrx, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 4.0)

hl, wl, = img1.shape[:2]
hr, wr, = img2.shape[:2]

result = cv2.warpPerspective(img2, mtrx, (wr+wl, hr))
result[0:hl, 0:wl] = img1

plt.figure(figsize=(20,20)),plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB)),plt.show()